In [29]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

# Parameters Config

In [30]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [56]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [57]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [58]:
ctable = CharacterTable(chars)

In [59]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# 加法器

# Data Generation (adder)

In [35]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [36]:
print(questions[:5], expected[:5])

['5+97   ', '41+380 ', '83+896 ', '9+547  ', '715+94 '] ['102 ', '421 ', '979 ', '556 ', '809 ']


# Processing

In [37]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [38]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [39]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False False False  True False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False False False False False False False False  True
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False False False False False  Tr

# Build Model

In [40]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [41]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 236us/step - loss: 2.0145 - acc: 0.2880 - val_loss: 1.8709 - val_acc: 0.3275
Q 782+62  T 844  ☒ 189 
Q 170+69  T 239  ☒ 158 
Q 689+53  T 742  ☒ 118 
Q 707+813 T 1520 ☒ 110 
Q 439+276 T 715  ☒ 110 
Q 797+26  T 823  ☒ 118 
Q 104+538 T 642  ☒ 118 
Q 24+304  T 328  ☒ 159 
Q 995+27  T 1022 ☒ 118 
Q 53+132  T 185  ☒ 159 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 160us/step - loss: 1.8501 - acc: 0.3269 - val_loss: 1.8347 - val_acc: 0.3286
Q 294+668 T 962  ☒ 100 
Q 656+776 T 1432 ☒ 101 
Q 602+246 T 848  ☒ 100 
Q 381+84  T 465  ☒ 107 
Q 356+320 T 676  ☒ 101 
Q 329+864 T 1193 ☒ 101 
Q 740+8   T 748  ☒ 17  
Q 86+69   T 155  ☒ 107 
Q 417+78  T 495  ☒ 107 
Q 58+178  T 236  ☒ 107 

-------------------

18000/18000 [==============================] - 3s 164us/step - loss: 1.2258 - acc: 0.5460 - val_loss: 1.2185 - val_acc: 0.5430
Q 42+517  T 559  ☒ 556 
Q 405+51  T 456  ☒ 495 
Q 534+264 T 798  ☒ 896 
Q 842+82  T 924  ☒ 965 
Q 174+99  T 273  ☒ 276 
Q 85+5    T 90   ☒ 15  
Q 122+28  T 150  ☒ 245 
Q 43+644  T 687  ☑ 687 
Q 88+565  T 653  ☒ 645 
Q 588+4   T 592  ☒ 595 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 168us/step - loss: 1.1869 - acc: 0.5606 - val_loss: 1.1818 - val_acc: 0.5582
Q 557+469 T 1026 ☒ 1012
Q 85+777  T 862  ☒ 855 
Q 97+798  T 895  ☒ 877 
Q 386+43  T 429  ☒ 412 
Q 6+708   T 714  ☒ 616 
Q 303+67  T 370  ☒ 322 
Q 929+333 T 1262 ☒ 1276
Q 597+65  T 662  ☒ 642 
Q 561+22  T 583  ☒ 576 
Q 359+6   T 365  ☑ 365 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 165us/step - loss: 0.1280 - acc: 0.9740 - val_loss: 0.1646 - val_acc: 0.9519
Q 77+568  T 645  ☑ 645 
Q 297+70  T 367  ☑ 367 
Q 32+963  T 995  ☑ 995 
Q 701+401 T 1102 ☒ 1103
Q 410+328 T 738  ☑ 738 
Q 34+13   T 47   ☒ 56  
Q 3+390   T 393  ☑ 393 
Q 539+692 T 1231 ☑ 1231
Q 258+42  T 300  ☑ 300 
Q 110+423 T 533  ☑ 533 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 199us/step - loss: 0.1204 - acc: 0.9742 - val_loss: 0.1528 - val_acc: 0.9544
Q 18+997  T 1015 ☑ 1015
Q 50+577  T 627  ☑ 627 
Q 62+57   T 119  ☑ 119 
Q 13+246  T 259  ☑ 259 
Q 3+592   T 595  ☑ 595 
Q 407+659 T 1066 ☑ 1066
Q 347+508 T 855  ☑ 855 
Q 440+10  T 450  ☑ 450 
Q 42+34   T 76   ☑ 76  
Q 95+367  T 462  ☑ 462 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 165us/step - loss: 0.0238 - acc: 0.9968 - val_loss: 0.0565 - val_acc: 0.9810
Q 43+644  T 687  ☑ 687 
Q 1+813   T 814  ☑ 814 
Q 60+391  T 451  ☑ 451 
Q 53+29   T 82   ☑ 82  
Q 34+644  T 678  ☑ 678 
Q 556+744 T 1300 ☒ 1390
Q 649+552 T 1201 ☒ 1101
Q 237+4   T 241  ☑ 241 
Q 815+120 T 935  ☑ 935 
Q 80+19   T 99   ☒ 909 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 166us/step - loss: 0.0157 - acc: 0.9992 - val_loss: 0.0509 - val_acc: 0.9852
Q 351+827 T 1178 ☑ 1178
Q 635+957 T 1592 ☑ 1592
Q 644+31  T 675  ☑ 675 
Q 423+682 T 1105 ☑ 1105
Q 516+45  T 561  ☑ 561 
Q 52+105  T 157  ☑ 157 
Q 67+470  T 537  ☑ 537 
Q 52+6    T 58   ☑ 58  
Q 682+297 T 979  ☑ 979 
Q 51+750  T 801  ☑ 801 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 166us/step - loss: 0.0977 - acc: 0.9662 - val_loss: 0.1267 - val_acc: 0.9555
Q 401+876 T 1277 ☑ 1277
Q 84+708  T 792  ☑ 792 
Q 0+929   T 929  ☒ 939 
Q 555+131 T 686  ☑ 686 
Q 15+659  T 674  ☑ 674 
Q 308+85  T 393  ☑ 393 
Q 216+353 T 569  ☒ 560 
Q 379+175 T 554  ☑ 554 
Q 703+67  T 770  ☑ 770 
Q 999+93  T 1092 ☑ 1092

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 163us/step - loss: 0.0311 - acc: 0.9912 - val_loss: 0.0518 - val_acc: 0.9826
Q 620+13  T 633  ☑ 633 
Q 6+650   T 656  ☑ 656 
Q 10+632  T 642  ☑ 642 
Q 895+83  T 978  ☑ 978 
Q 54+782  T 836  ☑ 836 
Q 524+485 T 1009 ☑ 1009
Q 762+176 T 938  ☑ 938 
Q 50+143  T 193  ☑ 193 
Q 930+111 T 1041 ☑ 1041
Q 967+0   T 967  ☑ 967 


# Testing

In [42]:
print("MSG : Prediction")

MSG : Prediction


In [52]:
for i in range(10):
        ind = np.random.randint(0, len(test_x))
        rowx, rowy = test_x[np.array([ind])], test_y[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Q 468+407 T 875  ☑ 875 
Q 23+855  T 878  ☑ 878 
Q 386+57  T 443  ☑ 443 
Q 91+552  T 643  ☑ 643 
Q 490+8   T 498  ☑ 498 
Q 31+402  T 433  ☑ 433 
Q 455+166 T 621  ☑ 621 
Q 1+829   T 830  ☑ 830 
Q 67+11   T 78   ☑ 78  
Q 539+38  T 577  ☑ 577 


# 減法器

# Data Generation (subtractor)

In [53]:
questions1 = []
expected1 = []
seen1 = set()
print('Generating data...')
while len(questions1) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen1:
        continue
    seen1.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    #if REVERSE:
    #    query = query[::-1]
    questions1.append(query)
    expected1.append(ans)
print('Total addition questions:', len(questions1))

Generating data...
Total addition questions: 80000


In [54]:
print(questions1[:5], expected1[:5])

['0-87   ', '752-677', '4-8    ', '61-3   ', '7-54   '] ['-87 ', '75  ', '-4  ', '58  ', '-47 ']


# Processing

In [60]:
print('Vectorization...')
x1 = np.zeros((len(questions1), MAXLEN, len(chars)), dtype=np.bool)
y1 = np.zeros((len(expected1), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions1):
    x1[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected1):
    y1[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [66]:
indices1 = np.arange(len(y1))
np.random.shuffle(indices1)
x1 = x1[indices1]
y1 = y1[indices1]

# train_test_split
train_x1 = x1[:50000]
train_y1 = y1[:50000]
test_x1 = x1[50000:]
test_y1 = y1[50000:]

split_at = len(train_x1) - len(train_x1) // 10
(x_train1, x_val1) = train_x1[:split_at], train_x1[split_at:]
(y_train1, y_val1) = train_y1[:split_at], train_y1[split_at:]

print('Training Data:')
print(x_train1.shape)
print(y_train1.shape)

print('Validation Data:')
print(x_val1.shape)
print(y_val1.shape)

print('Testing Data:')
print(test_x1.shape)
print(test_y1.shape)

Training Data:
(45000, 7, 13)
(45000, 4, 13)
Validation Data:
(5000, 7, 13)
(5000, 4, 13)
Testing Data:
(30000, 7, 13)
(30000, 4, 13)


# Build Model

In [67]:
print('Build model...')

model1 = Sequential()
model1.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model1.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model1.add(RNN(HIDDEN_SIZE, return_sequences=True))

model1.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model1.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [68]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model1.fit(x_train1, y_train1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val1, y_val1))
    for i in range(10):
        ind = np.random.randint(0, len(x_val1))
        rowx, rowy = x_val1[np.array([ind])], y_val1[np.array([ind])]
        preds = model1.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 206us/step - loss: 1.9395 - acc: 0.3199 - val_loss: 1.6970 - val_acc: 0.3765
Q 977-28  T 949  ☒ 888 
Q 230-850 T -620 ☒ -116
Q 2-645   T -643 ☒ -215
Q 761-51  T 710  ☒ 450 
Q 61-614  T -553 ☒ -111
Q 87-254  T -167 ☒ -111
Q 242-579 T -337 ☒ -11 
Q 90-297  T -207 ☒ -607
Q 824-56  T 768  ☒ 440 
Q 4-991   T -987 ☒ -888

--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 171us/step - loss: 1.6445 - acc: 0.3933 - val_loss: 1.5830 - val_acc: 0.4214
Q 138-86  T 52   ☒ -1  
Q 665-461 T 204  ☒ 11  
Q 58-291  T -233 ☒ -300
Q 334-497 T -163 ☒ -22 
Q 979-975 T 4    ☒ 11  
Q 34-21   T 13   ☒ 11  
Q 42-759  T -717 ☒ -700
Q 68-444  T -376 ☒ -300
Q 40-465  T -425 ☒ -400
Q 588-31  T 557  ☒ 768 

-------------------

45000/45000 [==============================] - 8s 188us/step - loss: 0.9355 - acc: 0.6542 - val_loss: 0.9347 - val_acc: 0.6468
Q 366-34  T 332  ☑ 332 
Q 185-6   T 179  ☒ 176 
Q 676-22  T 654  ☒ 667 
Q 44-77   T -33  ☒ -32 
Q 40-805  T -765 ☒ -761
Q 633-80  T 553  ☒ 540 
Q 620-693 T -73  ☒ -36 
Q 473-30  T 443  ☒ 442 
Q 177-816 T -639 ☒ -657
Q 389-4   T 385  ☑ 385 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 193us/step - loss: 0.8986 - acc: 0.6687 - val_loss: 0.8974 - val_acc: 0.6621
Q 613-87  T 526  ☒ 525 
Q 880-99  T 781  ☒ 799 
Q 336-42  T 294  ☒ 292 
Q 20-13   T 7    ☒ 1   
Q 717-37  T 680  ☒ 682 
Q 504-256 T 248  ☒ 225 
Q 370-543 T -173 ☒ -172
Q 851-24  T 827  ☒ 821 
Q 408-606 T -198 ☒ -111
Q 511-539 T -28  ☒ -10 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 165us/step - loss: 0.3497 - acc: 0.8867 - val_loss: 0.3569 - val_acc: 0.8814
Q 0-382   T -382 ☒ -383
Q 206-58  T 148  ☑ 148 
Q 52-736  T -684 ☑ -684
Q 64-375  T -311 ☑ -311
Q 89-803  T -714 ☑ -714
Q 68-370  T -302 ☑ -302
Q 878-15  T 863  ☑ 863 
Q 99-670  T -571 ☑ -571
Q 20-539  T -519 ☒ -518
Q 819-467 T 352  ☑ 352 

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 166us/step - loss: 0.3131 - acc: 0.9020 - val_loss: 0.3061 - val_acc: 0.9003
Q 354-990 T -636 ☑ -636
Q 9-305   T -296 ☑ -296
Q 6-592   T -586 ☑ -586
Q 76-72   T 4    ☒ 15  
Q 2-508   T -506 ☑ -506
Q 334-13  T 321  ☑ 321 
Q 808-343 T 465  ☒ 455 
Q 148-492 T -344 ☑ -344
Q 211-89  T 122  ☒ 123 
Q 916-738 T 178  ☒ 188 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 165us/step - loss: 0.0997 - acc: 0.9723 - val_loss: 0.1098 - val_acc: 0.9658
Q 575-309 T 266  ☑ 266 
Q 442-96  T 346  ☑ 346 
Q 474-26  T 448  ☑ 448 
Q 50-5    T 45   ☑ 45  
Q 519-46  T 473  ☑ 473 
Q 579-83  T 496  ☑ 496 
Q 195-72  T 123  ☑ 123 
Q 670-45  T 625  ☑ 625 
Q 63-856  T -793 ☑ -793
Q 61-526  T -465 ☑ -465

--------------------------------------------------
Iteration 43
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 169us/step - loss: 0.0885 - acc: 0.9765 - val_loss: 0.1590 - val_acc: 0.9494
Q 36-293  T -257 ☑ -257
Q 260-127 T 133  ☑ 133 
Q 13-788  T -775 ☑ -775
Q 540-821 T -281 ☑ -281
Q 717-42  T 675  ☑ 675 
Q 49-9    T 40   ☑ 40  
Q 32-404  T -372 ☑ -372
Q 724-80  T 644  ☑ 644 
Q 854-28  T 826  ☑ 826 
Q 68-722  T -654 ☑ -654

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 8s 187us/step - loss: 0.0637 - acc: 0.9817 - val_loss: 0.0636 - val_acc: 0.9793
Q 407-7   T 400  ☑ 400 
Q 647-795 T -148 ☑ -148
Q 240-80  T 160  ☑ 160 
Q 614-848 T -234 ☑ -234
Q 76-31   T 45   ☑ 45  
Q 95-724  T -629 ☑ -629
Q 411-244 T 167  ☑ 167 
Q 951-403 T 548  ☑ 548 
Q 123-6   T 117  ☑ 117 
Q 184-8   T 176  ☑ 176 

--------------------------------------------------
Iteration 57
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 186us/step - loss: 0.0334 - acc: 0.9933 - val_loss: 0.0495 - val_acc: 0.9845
Q 716-721 T -5   ☑ -5  
Q 930-747 T 183  ☑ 183 
Q 680-708 T -28  ☑ -28 
Q 5-296   T -291 ☑ -291
Q 139-19  T 120  ☒ 110 
Q 372-87  T 285  ☑ 285 
Q 905-88  T 817  ☑ 817 
Q 479-896 T -417 ☑ -417
Q 93-860  T -767 ☑ -767
Q 488-37  T 451  ☑ 451 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 9s 197us/step - loss: 0.0283 - acc: 0.9934 - val_loss: 0.0632 - val_acc: 0.9787
Q 254-629 T -375 ☑ -375
Q 896-378 T 518  ☑ 518 
Q 981-937 T 44   ☑ 44  
Q 83-214  T -131 ☑ -131
Q 118-708 T -590 ☑ -590
Q 191-9   T 182  ☑ 182 
Q 200-797 T -597 ☒ -497
Q 169-370 T -201 ☒ -101
Q 390-3   T 387  ☑ 387 
Q 878-15  T 863  ☑ 863 

--------------------------------------------------
Iteration 71
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 0.0499 - acc: 0.9875 - val_loss: 0.2858 - val_acc: 0.9185
Q 46-794  T -748 ☒ -747
Q 8-84    T -76  ☑ -76 
Q 234-99  T 135  ☒ 234 
Q 655-38  T 617  ☑ 617 
Q 964-45  T 919  ☑ 919 
Q 335-146 T 189  ☑ 189 
Q 975-80  T 895  ☑ 895 
Q 81-587  T -506 ☑ -506
Q 70-42   T 28   ☒ 18  
Q 35-248  T -213 ☑ -213

--------------------------------------------------
Iteration 72
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 160us/step - loss: 0.0099 - acc: 0.9989 - val_loss: 0.0265 - val_acc: 0.9913
Q 825-53  T 772  ☑ 772 
Q 7-450   T -443 ☑ -443
Q 447-191 T 256  ☑ 256 
Q 51-369  T -318 ☑ -318
Q 41-210  T -169 ☑ -169
Q 171-38  T 133  ☑ 133 
Q 547-68  T 479  ☑ 479 
Q 941-62  T 879  ☑ 879 
Q 27-204  T -177 ☑ -177
Q 13-566  T -553 ☑ -553

--------------------------------------------------
Iteration 85
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 159us/step - loss: 0.0374 - acc: 0.9897 - val_loss: 0.2233 - val_acc: 0.9327
Q 410-467 T -57  ☒ -47 
Q 112-69  T 43   ☑ 43  
Q 411-244 T 167  ☑ 167 
Q 227-341 T -114 ☑ -114
Q 946-58  T 888  ☑ 888 
Q 5-63    T -58  ☒ -68 
Q 420-18  T 402  ☑ 402 
Q 568-89  T 479  ☑ 479 
Q 929-76  T 853  ☑ 853 
Q 508-51  T 457  ☑ 457 

--------------------------------------------------
Iteration 86
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

45000/45000 [==============================] - 7s 160us/step - loss: 0.0083 - acc: 0.9991 - val_loss: 0.0235 - val_acc: 0.9921
Q 220-722 T -502 ☑ -502
Q 671-219 T 452  ☑ 452 
Q 390-89  T 301  ☒ 201 
Q 9-548   T -539 ☑ -539
Q 47-89   T -42  ☑ -42 
Q 279-17  T 262  ☑ 262 
Q 0-560   T -560 ☑ -560
Q 960-733 T 227  ☑ 227 
Q 864-70  T 794  ☑ 794 
Q 659-794 T -135 ☑ -135

--------------------------------------------------
Iteration 99
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 163us/step - loss: 0.0069 - acc: 0.9994 - val_loss: 0.0228 - val_acc: 0.9922
Q 225-420 T -195 ☑ -195
Q 968-71  T 897  ☑ 897 
Q 302-6   T 296  ☑ 296 
Q 2-493   T -491 ☑ -491
Q 0-419   T -419 ☑ -419
Q 318-62  T 256  ☑ 256 
Q 323-445 T -122 ☑ -122
Q 375-855 T -480 ☑ -480
Q 36-452  T -416 ☑ -416
Q 369-68  T 301  ☑ 301 


# Testing

In [72]:
preds = model1.predict_classes(test_x1)

count = 0
for i in range(len(preds)):
    
    q = ctable.decode(test_x1[i])
    correct = ctable.decode(test_y1[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    
print(count/len(preds))

Q 361-1   T 360  ☑ 360 
Q 749-427 T 322  ☑ 322 
Q 165-900 T -735 ☑ -735
Q 214-889 T -675 ☑ -675
Q 274-21  T 253  ☑ 253 
Q 131-381 T -250 ☑ -250
Q 7-346   T -339 ☑ -339
Q 120-54  T 66   ☑ 66  
Q 717-18  T 699  ☑ 699 
Q 120-57  T 63   ☑ 63  
Q 74-166  T -92  ☑ -92 
Q 797-216 T 581  ☑ 581 
Q 756-345 T 411  ☑ 411 
Q 673-740 T -67  ☑ -67 
Q 85-73   T 12   ☑ 12  
Q 511-99  T 412  ☑ 412 
Q 1-810   T -809 ☑ -809
Q 592-464 T 128  ☑ 128 
Q 44-164  T -120 ☑ -120
Q 617-145 T 472  ☑ 472 
Q 578-592 T -14  ☑ -14 
Q 991-18  T 973  ☑ 973 
Q 243-528 T -285 ☑ -285
Q 751-62  T 689  ☑ 689 
Q 365-867 T -502 ☑ -502
Q 28-18   T 10   ☑ 10  
Q 72-191  T -119 ☑ -119
Q 657-94  T 563  ☑ 563 
Q 260-433 T -173 ☑ -173
Q 66-167  T -101 ☑ -101
Q 9-975   T -966 ☑ -966
Q 710-466 T 244  ☑ 244 
Q 74-16   T 58   ☑ 58  
Q 484-60  T 424  ☑ 424 
Q 914-97  T 817  ☑ 817 
Q 472-69  T 403  ☑ 403 
Q 912-779 T 133  ☑ 133 
Q 75-560  T -485 ☑ -485
Q 17-215  T -198 ☑ -198
Q 336-3   T 333  ☑ 333 
Q 68-729  T -661 ☑ -661
Q 101-90  T 11  

Q 818-95  T 723  ☑ 723 
Q 8-145   T -137 ☑ -137
Q 4-92    T -88  ☑ -88 
Q 14-260  T -246 ☑ -246
Q 606-742 T -136 ☑ -136
Q 975-63  T 912  ☑ 912 
Q 36-924  T -888 ☑ -888
Q 482-621 T -139 ☑ -139
Q 412-97  T 315  ☑ 315 
Q 70-665  T -595 ☑ -595
Q 46-25   T 21   ☑ 21  
Q 607-637 T -30  ☑ -30 
Q 850-404 T 446  ☑ 446 
Q 325-3   T 322  ☑ 322 
Q 666-813 T -147 ☑ -147
Q 684-62  T 622  ☑ 622 
Q 553-4   T 549  ☑ 549 
Q 368-289 T 79   ☑ 79  
Q 763-701 T 62   ☑ 62  
Q 163-24  T 139  ☑ 139 
Q 443-95  T 348  ☑ 348 
Q 3-418   T -415 ☑ -415
Q 765-466 T 299  ☑ 299 
Q 740-277 T 463  ☑ 463 
Q 97-207  T -110 ☑ -110
Q 39-260  T -221 ☑ -221
Q 807-99  T 708  ☑ 708 
Q 16-258  T -242 ☑ -242
Q 7-209   T -202 ☑ -202
Q 803-688 T 115  ☑ 115 
Q 636-69  T 567  ☑ 567 
Q 61-584  T -523 ☑ -523
Q 420-974 T -554 ☑ -554
Q 217-557 T -340 ☑ -340
Q 11-612  T -601 ☑ -601
Q 390-17  T 373  ☑ 373 
Q 39-346  T -307 ☑ -307
Q 207-1   T 206  ☑ 206 
Q 38-169  T -131 ☑ -131
Q 29-342  T -313 ☑ -313
Q 377-9   T 368  ☑ 368 
Q 300-372 T -72 

Q 917-237 T 680  ☒ 670 
Q 1-798   T -797 ☑ -797
Q 69-186  T -117 ☑ -117
Q 873-750 T 123  ☑ 123 
Q 82-657  T -575 ☑ -575
Q 970-84  T 886  ☑ 886 
Q 586-387 T 199  ☑ 199 
Q 498-461 T 37   ☑ 37  
Q 725-676 T 49   ☑ 49  
Q 926-55  T 871  ☑ 871 
Q 31-25   T 6    ☑ 6   
Q 170-16  T 154  ☑ 154 
Q 593-62  T 531  ☑ 531 
Q 694-59  T 635  ☑ 635 
Q 67-831  T -764 ☑ -764
Q 773-4   T 769  ☑ 769 
Q 1-158   T -157 ☑ -157
Q 357-993 T -636 ☑ -636
Q 605-67  T 538  ☑ 538 
Q 896-619 T 277  ☑ 277 
Q 551-123 T 428  ☑ 428 
Q 522-24  T 498  ☑ 498 
Q 532-515 T 17   ☑ 17  
Q 393-72  T 321  ☑ 321 
Q 959-21  T 938  ☑ 938 
Q 857-5   T 852  ☑ 852 
Q 79-318  T -239 ☑ -239
Q 676-434 T 242  ☑ 242 
Q 671-731 T -60  ☑ -60 
Q 818-800 T 18   ☑ 18  
Q 10-223  T -213 ☑ -213
Q 70-41   T 29   ☑ 29  
Q 413-28  T 385  ☑ 385 
Q 611-997 T -386 ☑ -386
Q 782-111 T 671  ☑ 671 
Q 26-478  T -452 ☑ -452
Q 916-853 T 63   ☑ 63  
Q 373-90  T 283  ☑ 283 
Q 671-128 T 543  ☑ 543 
Q 450-36  T 414  ☑ 414 
Q 953-566 T 387  ☑ 387 
Q 115-747 T -632

Q 995-369 T 626  ☑ 626 
Q 256-700 T -444 ☑ -444
Q 937-36  T 901  ☑ 901 
Q 994-87  T 907  ☑ 907 
Q 965-587 T 378  ☑ 378 
Q 50-226  T -176 ☑ -176
Q 321-16  T 305  ☑ 305 
Q 456-158 T 298  ☑ 298 
Q 806-12  T 794  ☑ 794 
Q 37-482  T -445 ☑ -445
Q 36-115  T -79  ☑ -79 
Q 990-40  T 950  ☑ 950 
Q 32-891  T -859 ☑ -859
Q 51-8    T 43   ☑ 43  
Q 601-884 T -283 ☑ -283
Q 95-65   T 30   ☑ 30  
Q 413-12  T 401  ☑ 401 
Q 428-600 T -172 ☑ -172
Q 586-504 T 82   ☑ 82  
Q 950-238 T 712  ☑ 712 
Q 3-472   T -469 ☑ -469
Q 615-42  T 573  ☑ 573 
Q 10-756  T -746 ☑ -746
Q 712-79  T 633  ☑ 633 
Q 1-350   T -349 ☑ -349
Q 73-648  T -575 ☑ -575
Q 7-535   T -528 ☑ -528
Q 69-363  T -294 ☑ -294
Q 855-6   T 849  ☑ 849 
Q 157-47  T 110  ☑ 110 
Q 66-363  T -297 ☑ -297
Q 930-28  T 902  ☑ 902 
Q 920-48  T 872  ☑ 872 
Q 932-19  T 913  ☑ 913 
Q 943-991 T -48  ☑ -48 
Q 344-13  T 331  ☑ 331 
Q 424-3   T 421  ☑ 421 
Q 500-31  T 469  ☑ 469 
Q 781-669 T 112  ☑ 112 
Q 180-966 T -786 ☑ -786
Q 14-470  T -456 ☑ -456
Q 36-668  T -632

Q 6-11    T -5   ☑ -5  
Q 80-933  T -853 ☑ -853
Q 155-516 T -361 ☑ -361
Q 653-985 T -332 ☑ -332
Q 46-994  T -948 ☑ -948
Q 733-258 T 475  ☑ 475 
Q 224-6   T 218  ☑ 218 
Q 310-183 T 127  ☑ 127 
Q 397-27  T 370  ☑ 370 
Q 62-730  T -668 ☑ -668
Q 499-90  T 409  ☑ 409 
Q 979-52  T 927  ☑ 927 
Q 1-667   T -666 ☑ -666
Q 12-337  T -325 ☑ -325
Q 262-994 T -732 ☑ -732
Q 2-264   T -262 ☑ -262
Q 26-543  T -517 ☑ -517
Q 168-953 T -785 ☑ -785
Q 522-596 T -74  ☑ -74 
Q 11-586  T -575 ☑ -575
Q 2-249   T -247 ☑ -247
Q 78-332  T -254 ☑ -254
Q 60-849  T -789 ☑ -789
Q 730-1   T 729  ☑ 729 
Q 63-777  T -714 ☑ -714
Q 796-580 T 216  ☑ 216 
Q 61-663  T -602 ☑ -602
Q 80-541  T -461 ☑ -461
Q 187-275 T -88  ☑ -88 
Q 418-79  T 339  ☑ 339 
Q 586-63  T 523  ☑ 523 
Q 83-673  T -590 ☑ -590
Q 959-2   T 957  ☑ 957 
Q 78-991  T -913 ☑ -913
Q 650-1   T 649  ☑ 649 
Q 689-85  T 604  ☑ 604 
Q 26-949  T -923 ☑ -923
Q 462-262 T 200  ☑ 200 
Q 809-361 T 448  ☑ 448 
Q 826-8   T 818  ☑ 818 
Q 38-198  T -160 ☑ -160
Q 24-582  T -558

...

Q 728-294 T 434  ☑ 434 
Q 659-537 T 122  ☑ 122 
Q 143-574 T -431 ☑ -431
Q 891-37  T 854  ☑ 854 
Q 358-18  T 340  ☒ 330 
Q 937-268 T 669  ☑ 669 
Q 992-86  T 906  ☑ 906 
Q 40-586  T -546 ☑ -546
Q 822-274 T 548  ☑ 548 
Q 878-217 T 661  ☑ 661 
Q 83-65   T 18   ☑ 18  
Q 258-174 T 84   ☑ 84  
Q 209-869 T -660 ☑ -660
Q 6-727   T -721 ☑ -721
Q 269-84  T 185  ☑ 185 
Q 797-1   T 796  ☑ 796 
Q 84-179  T -95  ☑ -95 
Q 23-747  T -724 ☑ -724
Q 634-403 T 231  ☑ 231 
Q 689-149 T 540  ☑ 540 
Q 831-44  T 787  ☑ 787 
Q 501-610 T -109 ☑ -109
Q 560-41  T 519  ☑ 519 
Q 962-29  T 933  ☑ 933 
Q 674-46  T 628  ☑ 628 
Q 23-467  T -444 ☑ -444
Q 931-232 T 699  ☒ 609 
Q 72-215  T -143 ☑ -143
Q 920-78  T 842  ☑ 842 
Q 895-869 T 26   ☑ 26  
Q 54-643  T -589 ☑ -589
Q 31-867  T -836 ☑ -836
Q 618-272 T 346  ☑ 346 
Q 164-260 T -96  ☒ -16 
Q 84-14   T 70   ☑ 70  
Q 849-214 T 635  ☑ 635 
Q 6-8     T -2   ☑ -2  
Q 748-73  T 675  ☑ 675 
Q 471-17  T 454  ☑ 454 
Q 112-83  T 29   ☒ 39  
Q 964-3   T 961  ☑ 961 
Q 130-254 T -124

Q 88-47   T 41   ☑ 41  
Q 554-276 T 278  ☑ 278 
Q 308-280 T 28   ☑ 28  
Q 552-9   T 543  ☑ 543 
Q 973-5   T 968  ☑ 968 
Q 429-77  T 352  ☑ 352 
Q 77-374  T -297 ☑ -297
Q 790-930 T -140 ☑ -140
Q 91-887  T -796 ☑ -796
Q 245-63  T 182  ☑ 182 
Q 428-75  T 353  ☑ 353 
Q 621-30  T 591  ☑ 591 
Q 249-285 T -36  ☑ -36 
Q 212-5   T 207  ☑ 207 
Q 226-991 T -765 ☑ -765
Q 26-2    T 24   ☑ 24  
Q 12-514  T -502 ☑ -502
Q 862-59  T 803  ☑ 803 
Q 18-823  T -805 ☑ -805
Q 745-2   T 743  ☑ 743 
Q 11-900  T -889 ☑ -889
Q 148-729 T -581 ☑ -581
Q 725-287 T 438  ☑ 438 
Q 52-799  T -747 ☑ -747
Q 842-981 T -139 ☑ -139
Q 698-148 T 550  ☑ 550 
Q 407-520 T -113 ☑ -113
Q 488-259 T 229  ☑ 229 
Q 687-425 T 262  ☑ 262 
Q 186-993 T -807 ☑ -807
Q 726-737 T -11  ☑ -11 
Q 641-132 T 509  ☑ 509 
Q 70-826  T -756 ☑ -756
Q 8-869   T -861 ☑ -861
Q 684-180 T 504  ☑ 504 
Q 207-907 T -700 ☑ -700
Q 16-564  T -548 ☑ -548
Q 38-820  T -782 ☑ -782
Q 347-65  T 282  ☑ 282 
Q 881-70  T 811  ☑ 811 
Q 22-848  T -826 ☑ -826
Q 769-991 T -222

Q 333-34  T 299  ☑ 299 
Q 392-26  T 366  ☑ 366 
Q 69-949  T -880 ☑ -880
Q 15-621  T -606 ☑ -606
Q 178-319 T -141 ☑ -141
Q 313-68  T 245  ☑ 245 
Q 329-301 T 28   ☒ 18  
Q 431-478 T -47  ☑ -47 
Q 49-399  T -350 ☑ -350
Q 496-699 T -203 ☑ -203
Q 513-287 T 226  ☑ 226 
Q 368-691 T -323 ☑ -323
Q 841-65  T 776  ☑ 776 
Q 682-583 T 99   ☑ 99  
Q 404-77  T 327  ☑ 327 
Q 505-809 T -304 ☑ -304
Q 27-44   T -17  ☑ -17 
Q 427-104 T 323  ☑ 323 
Q 135-53  T 82   ☑ 82  
Q 76-673  T -597 ☑ -597
Q 738-5   T 733  ☑ 733 
Q 999-386 T 613  ☑ 613 
Q 911-548 T 363  ☑ 363 
Q 980-908 T 72   ☒ 62  
Q 245-948 T -703 ☑ -703
Q 496-676 T -180 ☑ -180
Q 328-42  T 286  ☑ 286 
Q 341-43  T 298  ☑ 298 
Q 582-663 T -81  ☑ -81 
Q 409-230 T 179  ☒ 189 
Q 89-901  T -812 ☑ -812
Q 819-550 T 269  ☑ 269 
Q 48-383  T -335 ☑ -335
Q 957-713 T 244  ☑ 244 
Q 564-913 T -349 ☑ -349
Q 826-567 T 259  ☑ 259 
Q 756-885 T -129 ☑ -129
Q 18-188  T -170 ☑ -170
Q 587-990 T -403 ☑ -403
Q 976-954 T 22   ☑ 22  
Q 78-248  T -170 ☑ -170
Q 5-147   T -142

Q 23-336  T -313 ☑ -313
Q 624-66  T 558  ☑ 558 
Q 549-782 T -233 ☑ -233
Q 707-432 T 275  ☑ 275 
Q 259-28  T 231  ☑ 231 
Q 908-974 T -66  ☑ -66 
Q 619-389 T 230  ☑ 230 
Q 298-0   T 298  ☑ 298 
Q 162-24  T 138  ☑ 138 
Q 496-939 T -443 ☑ -443
Q 80-94   T -14  ☑ -14 
Q 635-54  T 581  ☑ 581 
Q 8-66    T -58  ☑ -58 
Q 23-739  T -716 ☑ -716
Q 78-6    T 72   ☑ 72  
Q 11-664  T -653 ☑ -653
Q 33-277  T -244 ☑ -244
Q 29-423  T -394 ☑ -394
Q 739-914 T -175 ☑ -175
Q 94-196  T -102 ☑ -102
Q 31-582  T -551 ☑ -551
Q 773-72  T 701  ☑ 701 
Q 746-450 T 296  ☑ 296 
Q 463-445 T 18   ☑ 18  
Q 75-760  T -685 ☑ -685
Q 809-529 T 280  ☑ 280 
Q 72-925  T -853 ☑ -853
Q 81-728  T -647 ☑ -647
Q 84-373  T -289 ☑ -289
Q 397-97  T 300  ☑ 300 
Q 965-362 T 603  ☑ 603 
Q 272-608 T -336 ☑ -336
Q 734-992 T -258 ☑ -258
Q 254-25  T 229  ☑ 229 
Q 57-852  T -795 ☑ -795
Q 213-819 T -606 ☒ -506
Q 787-52  T 735  ☑ 735 
Q 529-430 T 99   ☑ 99  
Q 326-121 T 205  ☑ 205 
Q 463-808 T -345 ☑ -345
Q 502-59  T 443  ☑ 443 
Q 217-45  T 172 

Q 78-657  T -579 ☑ -579
Q 959-388 T 571  ☑ 571 
Q 890-86  T 804  ☑ 804 
Q 671-7   T 664  ☑ 664 
Q 496-322 T 174  ☑ 174 
Q 7-457   T -450 ☑ -450
Q 207-515 T -308 ☑ -308
Q 159-186 T -27  ☑ -27 
Q 34-24   T 10   ☑ 10  
Q 81-276  T -195 ☑ -195
Q 39-117  T -78  ☑ -78 
Q 31-298  T -267 ☑ -267
Q 71-723  T -652 ☑ -652
Q 251-245 T 6    ☒ 7   
Q 11-615  T -604 ☑ -604
Q 961-87  T 874  ☑ 874 
Q 308-54  T 254  ☑ 254 
Q 29-879  T -850 ☑ -850
Q 387-579 T -192 ☑ -192
Q 969-765 T 204  ☑ 204 
Q 93-468  T -375 ☑ -375
Q 859-58  T 801  ☑ 801 
Q 741-123 T 618  ☑ 618 
Q 610-185 T 425  ☑ 425 
Q 405-991 T -586 ☑ -586
Q 404-111 T 293  ☑ 293 
Q 82-596  T -514 ☑ -514
Q 550-415 T 135  ☑ 135 
Q 616-97  T 519  ☑ 519 
Q 7-876   T -869 ☑ -869
Q 627-834 T -207 ☑ -207
Q 628-3   T 625  ☑ 625 
Q 620-89  T 531  ☑ 531 
Q 89-861  T -772 ☑ -772
Q 721-22  T 699  ☑ 699 
Q 630-136 T 494  ☑ 494 
Q 105-648 T -543 ☑ -543
Q 303-1   T 302  ☑ 302 
Q 185-712 T -527 ☑ -527
Q 151-81  T 70   ☑ 70  
Q 952-848 T 104  ☑ 104 
Q 404-977 T -573

In [76]:
print('the accuracy :')
print(count/len(preds))

the accuracy :
0.9735666666666667
